### visualize json files 

In [191]:
import os
import json
import pandas as pd

folder_path = "C:\\Users\\LENOVO\\test tanitai\\assignementdataset"
data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
            data.append(json_data)
            
df = pd.json_normalize(data)
print(df.head())
df.to_csv("C:\\Users\\LENOVO\\test tanitai\\combined_data.csv", index=False)


                                            paper_id  \
0  xml_output_grobid_python_client\-em-Fertility-...   
1  xml_output_grobid_python_client\A-&#x201c;firs...   
2  xml_output_grobid_python_client\A-behind-the-s...   
3  xml_output_grobid_python_client\A-call-to-acti...   
4  xml_output_grobid_python_client\A-case-report-...   

                                               title  \
0       Fertility and Sterility top videos from 2021   
1  A “first” on the horizon: the expansion of ute...   
2  A behind-the-scenes look at retroperitoneal ec...   
3  A call to action to reproductive endocrinologi...   
4  A case report of retroperitoneal ectopic pregn...   

                                             authors year venue  \
0  [{'first': 'Joseph', 'middle': [], 'last': 'Fi...       None   
1  [{'first': 'Elliott', 'middle': ['G'], 'last':...       None   
2  [{'first': 'Sarah', 'middle': [], 'last': 'Sim...       None   
3  [{'first': 'Nanette', 'middle': [], 'last': 'S...      

### Step2: Preprocessing Data 

In [192]:
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])


venue                                       50
pdf_parse.ref_entries.TABREF0.html          50
pdf_parse.ref_entries.TABREF0.num           50
pdf_parse.ref_entries.TABREF0.text          29
pdf_parse.ref_entries.TABREF0.content       29
                                            ..
pdf_parse.ref_entries.TABREF582.text        49
pdf_parse.ref_entries.TABREF582.html        50
pdf_parse.ref_entries.TABREF582.num         50
pdf_parse.ref_entries.TABREF582.content     49
pdf_parse.ref_entries.TABREF582.type_str    49
Length: 5518, dtype: int64


In [193]:
df = df[['paper_id', 'title', 'abstract', 'authors']]
print(df.head())

                                            paper_id  \
0  xml_output_grobid_python_client\-em-Fertility-...   
1  xml_output_grobid_python_client\A-&#x201c;firs...   
2  xml_output_grobid_python_client\A-behind-the-s...   
3  xml_output_grobid_python_client\A-call-to-acti...   
4  xml_output_grobid_python_client\A-case-report-...   

                                               title  \
0       Fertility and Sterility top videos from 2021   
1  A “first” on the horizon: the expansion of ute...   
2  A behind-the-scenes look at retroperitoneal ec...   
3  A call to action to reproductive endocrinologi...   
4  A case report of retroperitoneal ectopic pregn...   

                                            abstract  \
0  Objective: To objectively grade all video publ...   
1                                                      
2                                                      
3                                                      
4  Objective: To present a rare case of retrop

In [194]:

# Combine 'title' and 'abstract' into a single column called 'text'
df['text'] = df['title'] + " " + df['abstract']

# Drop the original 'title' and 'abstract' columns if no longer needed
df = df.drop(columns=['title', 'abstract'])

# Display the first few rows to verify
print(df.head())

                                            paper_id  \
0  xml_output_grobid_python_client\-em-Fertility-...   
1  xml_output_grobid_python_client\A-&#x201c;firs...   
2  xml_output_grobid_python_client\A-behind-the-s...   
3  xml_output_grobid_python_client\A-call-to-acti...   
4  xml_output_grobid_python_client\A-case-report-...   

                                             authors  \
0  [{'first': 'Joseph', 'middle': [], 'last': 'Fi...   
1  [{'first': 'Elliott', 'middle': ['G'], 'last':...   
2  [{'first': 'Sarah', 'middle': [], 'last': 'Sim...   
3  [{'first': 'Nanette', 'middle': [], 'last': 'S...   
4  [{'first': 'Jing', 'middle': [], 'last': 'Yuan...   

                                                text  
0  Fertility and Sterility top videos from 2021 O...  
1  A “first” on the horizon: the expansion of ute...  
2  A behind-the-scenes look at retroperitoneal ec...  
3  A call to action to reproductive endocrinologi...  
4  A case report of retroperitoneal ectopic pregn..

In [195]:
#cleaning data
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['text'] = df['text'].apply(clean_text)

In [196]:
df.tail()

,paper_id,authors,text
45,xml_output_cochrane\Boomsma_et_al-2019-Cochran...,"[{'first': 'Carolien', 'middle': ['M'], 'last'...",semen preparation techniques for intrauterine ...
46,xml_output_cochrane\Boomsma_et_al-2022-Cochran...,"[{'first': 'Carolien', 'middle': ['M'], 'last'...",periimplantation glucocorticoid administration...
47,xml_output_cochrane\Brito_et_al-2019-Cochrane_...,"[{'first': 'Luiz', 'middle': ['Gustavo'], 'las...",interventions for uterine fibroids an overview...
48,xml_output_cochrane\Bromley_et_al-2023-Cochran...,"[{'first': 'Rebecca', 'middle': [], 'last': 'B...",monotherapy treatment of epilepsy in pregnancy...
49,xml_output_cochrane\Brown_et_al-2016-Cochrane_...,"[{'first': 'Julie', 'middle': [], 'last': 'Bro...",ultrasound versus clinical touch for catheter ...


In [197]:
df.to_csv('preprocessed_data.csv')

In [198]:
#convert text to list 
def load_csv_file(path):
    df = pd.read_csv(path)
    return df['text'].tolist()

### Tokenization 

In [205]:
from transformers import BertTokenizer
#tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

path_to_csv = "preprocessed_data.csv"
X = load_csv_file(path_to_csv)

X_tokenized = tokenize_data(X)
y_tensor = torch.tensor(y)


In [200]:
# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X_tokenized['input_ids'], y_tensor, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Create DataLoader
train_dataset = TensorDataset(X_train.squeeze(1), y_train)
val_dataset = TensorDataset(X_val.squeeze(1), y_val)
test_dataset = TensorDataset(X_test.squeeze(1), y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [201]:
class TextClassifier(torch.nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids):
        output = self.bert(input_ids=input_ids)
        hidden_states = output.last_hidden_state
        output = self.dropout(hidden_states[:, 0, :])
        output = self.fc(output)
        return output

In [202]:
num_classes = len(torch.unique(y_tensor))
model = TextClassifier('bert-base-uncased', num_classes)

# Optimizer and Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


In [203]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu11


Looking in indexes: https://download.pytorch.org/whl/cu11
Note: you may need to restart the kernel to use updated packages.


###  Training

In [204]:
#Training
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            outputs = model(X_val_batch)
            loss = criterion(outputs, y_val_batch)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}')


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Evaluation

In [ ]:
#evaluation 
model.eval()
y_pred = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())

# Convert y_test to numpy array for comparison
y_true = y_test.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

### Save Model

In [ ]:
#Save the model 
model.save_pretrained('model.bin')
tokenizer.save_pretrained('modelTokenizer.json')
